# Module Import

In [ ]:
import numpy as np
import pandas as pd
import re
import argparse
import os
import pickle
import nltk
import tensorflow as tf
import re
import string
import ast
import tensorflow as tf
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    f1_score, auc, roc_curve, confusion_matrix)
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from gensim.models import KeyedVectors
from nltk import tokenize
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.utils.np_utils import to_categorical
from keras.layers import Layer
from keras import initializers
from keras import backend as K
from keras.layers import Dense, Input
from keras.layers import Embedding, GRU, Bidirectional,TimeDistributed,concatenate,LSTM
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from imblearn.over_sampling import RandomOverSampler,SMOTE
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.utils.np_utils import to_categorical
from keras.layers import Layer
from keras import initializers
from keras import backend as K
from keras.layers import Dense, Input
from keras.layers import Embedding, GRU, Bidirectional,TimeDistributed,concatenate,LSTM
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from imblearn.over_sampling import RandomOverSampler,SMOTE

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.compat.v1.Session(config=config)
os.environ['KERAS_BACKEND']='tensorflow'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
nltk.download('punkt')
nltk.download('stopwords')

# Method Loading

In [4]:
# partially from https://github.com/richliao/textClassifier/blob/master/textClassifierHATT.py

# removing URLs
def remove_url(text):
  url = re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r'', text)

# remove html tag
def remove_html(text):
  html = re.compile(r'<.*?>')
  return html.sub(r'', text)

# remove emoji
def remove_emoji(text):
  emoji_pattern = re.compile("["
  u"\U0001F600-\U0001F64F"  # emoticons
  u"\U0001F300-\U0001F5FF"  # symbols&pics
  u"\U0001F680-\U0001F6FF"  # transportation pic
  u"\U0001F1E0-\U0001F1FF"  # flags
  u"\U00002702-\U000027B0"
  u"\U000024C2-\U0001F251"
  "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', text)

# Stop Word Removal
NLTK_stop_words_list = stopwords.words('english')
custom_stop_words_list = ['...']
final_stop_words_list = NLTK_stop_words_list + custom_stop_words_list
def remove_stopwords(text):
  """custom function to remove the stopwords"""
  return " ".join([word for word in str(text).split() if word not in final_stop_words_list])

# Symbol removal
def clean_str(string):
  string = re.sub(r"[^A-Za-z0-9(),.!?\'\`]", " ", string)
  string = re.sub(r"\'s", " \'s", string)
  string = re.sub(r"\'ve", " \'ve", string)
  string = re.sub(r"\)", " \) ", string)
  string = re.sub(r"\?", " \? ", string)
  string = re.sub(r"\s{2,}", " ", string)
  string = re.sub(r"\\", "", string)
  string = re.sub(r"\'", "", string)
  string = re.sub(r"\"", "", string)
  return string.strip().lower()


def create_emb_mat(emb_path, word_idx, emb_dim):
    embeddings_index = {}
    if emb_file_flag == 'glove':
        f = open(os.path.join(embedding_path), encoding='utf-8')
        for line in f:
            values = line.split()
            word = values[0]
            vec = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vec
        f.close()
    else:
        wv_from_bin = KeyedVectors.load_word2vec_format(emb_path, limit=LIMIT)
        for word, vector in zip(wv_from_bin.vocab, wv_from_bin.vectors):
            vec = np.asarray(vector, dtype='float32')
            embeddings_index[word] = vec

    counter=0
    emb_matrix = np.random.random((len(word_idx) + 1, emb_dim))
    for word, i in word_idx.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            emb_matrix[i] = embedding_vector
        else :
            counter += 1
    return emb_matrix

class Attention(Layer):
    def __init__(self, attention_dim, **kwargs):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(Attention, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self._trainable_weights = [self.W, self.b, self.u]
        super(Attention, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, h, mask=None):
        uit = K.tanh(K.bias_add(K.dot(h, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = h * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

    def get_config(self):
        config = {
            'attention_dim': self.attention_dim
        }
        base_config = super(Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

# Data Loading

In [ ]:
!git clone https://github.com/anonymoususr12/MHPurf

In [ ]:
%cd MHPurf/data
# Setting Project
project = 'keras' # select in [tensorflow, pytorch, keras, incubator-mxnet, caffe]

path = f'{project}.csv'
pd_all = pd.read_csv(path)
pd_all = pd_all.sample(frac=1,random_state=999)
pd_all['Title+Body'] = ''
for idx in range(len(pd_all)):
  if pd.notna(pd_all['Body'].iloc[idx]):
    pd_all['Title+Body'].iloc[idx] = pd_all['Title'].iloc[idx] + '. ' + pd_all['Body'].iloc[idx]
  else:
    pd_all['Title+Body'].iloc[idx] = pd_all['Title'].iloc[idx]
pd_title = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Title":"text"})
pd_title.to_csv('Title.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_body = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Body":"text"})
pd_body.to_csv('Body.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_label = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Labels":"text"})
pd_label.to_csv('Labels.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_code = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Codes":"text"})
pd_code.to_csv('Codes.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_comment = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Comments":"text"})
pd_comment.to_csv('Comments.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_command = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Commands":"text"})
pd_command.to_csv('Command.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_tplusb = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Title+Body":"text"})
pd_tplusb.to_csv('Title+Body.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")

# Training

In [ ]:
# Imbalance Strategy Setting (if needed)
# IMBLEARN = 'RandomOverSampler'
# IMBLEARN = 'SMOTE'
IMBLEARN = 'None'

# RQ1 setting
PLUS_FLAG = True
TITLE_FLAG = False
DESC_FLAG = False

# RQ2 setting
COMMENT_FLAG = True
CODE_FLAG = True
COMMAND_FLAG = False
LABEL_FLAG = False

# Training setting
BATCH = 64
REPEAT = 30
EPOCH = 25
repeated_range = range(0,REPEAT)
# Embedding setting
WORD2VEC_FLAG = False # True for word2vec, false for GloVe
LIMIT = 1000000 # if WORD2VEC_FLAG

# Zero Padding Length Setting (if needed)
MAX_SENT_LENGTH = 150   # Zero padding length for word level vector (Max number of words in a sentence). We tried [50,100,150,200]
MAX_SENTS = 18  # Zero padding length for sentence level vector (Max number of sentences in a bug report). We tried [50,100,150,200]
MAX_NB_WORDS = 20000    # Maximum words in corpus We tried in [5000,10000,20000,4000]

# Comment feature reading method setting 
COMMENT_V2_FLAG = False # False for sentence version, True for comment version

# Generating output name
out_csv_name = f'../MHAN_{project}'
if COMMENT_V2_FLAG:
  out_csv_name += 'commentV2'
if WORD2VEC_FLAG == True:
  out_csv_name += '_word2vec' + str(LIMIT)
  EMBEDDING_PATH = "embedding/enwiki_20180420_100d.txt.bz2"
else:
  out_csv_name += '_glove'
  EMBEDDING_PATH = "embedding/glove.6B.100d.txt"
if COMMENT_FLAG == True:
  out_csv_name += '+comment'
if CODE_FLAG == True:
  out_csv_name += '+code'
if COMMAND_FLAG == True:
  out_csv_name += '+command'
if LABEL_FLAG == True:
  out_csv_name += '+label'
out_csv_name += '.csv'

tf_auc = tf.keras.metrics.AUC()
batch_size = BATCH
repeated_times = REPEAT
embedding_path = EMBEDDING_PATH

if embedding_path.find('glove') != -1:
    emb_file_flag = 'glove'     # pre-trained word vector is glove
    embedding_dim = int(((embedding_path.split('/')[-1]).split('.')[2])[:-1])
else:
  embedding_dim = 100
  print(f"glove is not using, set embedding_dim as {embedding_dim}")

#  preprocessing
###################### Preprocessing for comment #################################
if COMMENT_FLAG:  
  comment_train=pd.read_csv('Comments.csv')
  comment_train["text"] = comment_train["text"].fillna('')
  comments = []
  corpus3 = []

  if COMMENT_V2_FLAG:
  # # 2nd preprocessing method (comment based)
    for idx in range(comment_train.text.shape[0]):
      comment = ast.literal_eval(comment_train.text[idx])
      comment = list(map(remove_url,comment))
      comment = list(map(remove_html,comment))
      comment = list(map(remove_emoji,comment))
      comment = list(map(remove_stopwords,comment))
      comment = list(map(BeautifulSoup,comment))
      comment = [sometext.get_text() for sometext in comment]
      comment = list(map(clean_str,comment))
      comments.append(comment)
      comment = ''.join(comment)
      corpus3.append(comment) 

  else:
    # 1st preprocessing method (sentence based)
    for idx in range(comment_train.text.shape[0]):
      comment = ast.literal_eval(comment_train.text[idx])
      comment = list(map(remove_url,comment))
      comment = list(map(remove_html,comment))
      comment = list(map(remove_emoji,comment))
      comment = list(map(remove_stopwords,comment))
      comment = ''.join(comment) 
      comment = BeautifulSoup(comment)
      comment = clean_str(comment.get_text())
      corpus3.append(comment)

      comment = tokenize.sent_tokenize(comment)
      comments.append(comment)


  # Create Tokenizer
  tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(corpus3)


  # Zero Padding
  comment_data = np.zeros((len(corpus3), MAX_SENTS, MAX_SENT_LENGTH), dtype='int16')

  for i, sentences in enumerate(comments):
      for j, sent in enumerate(sentences):
          if j < MAX_SENTS:
              wordTokens = text_to_word_sequence(sent)
              k = 0
              for _, word in enumerate(wordTokens):
                  if word in tokenizer.word_index:
                    if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                        comment_data[i, j, k] = tokenizer.word_index[word]
                        k = k + 1

  comment_word_index = tokenizer.word_index

###################### Preprocessing for comment #################################

###################### Preprocessing for command #################################
if COMMAND_FLAG:  
  command_train=pd.read_csv('Command.csv')
  commands = []
  corpus4 = []

  for idx in range(command_train.text.shape[0]):
      command = ast.literal_eval(command_train.text[idx])
      command = list(map(remove_url,command))
      command = list(map(remove_html,command))
      command = list(map(remove_emoji,command))
      command = list(map(remove_stopwords,command))
      command = ''.join(command) 
      command = BeautifulSoup(command)
      command = clean_str(command.get_text())
      corpus4.append(command)
      command = tokenize.sent_tokenize(command)
      commands.append(command)
      
  # Create Tokenizer
  tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(corpus4)

  # Zero Padding
  command_data = np.zeros((len(corpus4), MAX_SENTS, MAX_SENT_LENGTH), dtype='int16')

  for i, sentences in enumerate(commands):
      for j, sent in enumerate(sentences):
          if j < MAX_SENTS:
              wordTokens = text_to_word_sequence(sent)
              k = 0
              for _, word in enumerate(wordTokens):
                  if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                      command_data[i, j, k] = tokenizer.word_index[word]
                      k = k + 1

  command_word_index = tokenizer.word_index

###################### Preprocessing for command #################################

###################### Preprocessing for Code #################################
if CODE_FLAG:  
  code_train=pd.read_csv('Codes.csv')
  codes = []
  corpus2 = []

  # 1st preprocess (line based)
  for idx in range(code_train.text.shape[0]):
    code = ast.literal_eval(code_train.text[idx])
    code = list(map(remove_url,code))
    code = list(map(remove_html,code))
    code = list(map(remove_emoji,code))
    code = list(map(remove_stopwords,code))
    code = ''.join(code) 
    code = BeautifulSoup(code)
    code = clean_str(code.get_text())
    corpus2.append(code)
    code = code.splitlines()
    code=list(reversed(code))
    codes.append(code) 

  # # 2nd preprocess (block based)
  # for idx in range(code_train.text.shape[0]):
  #     code = ast.literal_eval(code_train.text[idx])
  #     code = list(map(remove_url,code))
  #     code = list(map(remove_html,code))
  #     code = list(map(remove_emoji,code))
  #     code = list(map(remove_stopwords,code))
  #     code = list(map(BeautifulSoup,code))
  #     code = [sometext.get_text() for sometext in code]
  #     code = list(map(clean_str,code))
  #     codes.append(code)
  #     code = ''.join(code) 
  #     corpus2.append(code)
      
  # Create Tokenizer
  tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(corpus2)


  # Zero Padding
  code_data = np.zeros((len(corpus2), MAX_SENTS, MAX_SENT_LENGTH), dtype='int16')

  for i, sentences in enumerate(codes):
      for j, sent in enumerate(sentences):
          if j < MAX_SENTS:
              wordTokens = text_to_word_sequence(sent)
              k = 0
              for _, word in enumerate(wordTokens):
                if word in tokenizer.word_index:
                  if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                      code_data[i, j, k] = tokenizer.word_index[word]
                      k = k + 1

  code_word_index = tokenizer.word_index

###################### Preprocessing for Code #################################


###################### Preprocessing for Label #################################
if LABEL_FLAG:
  label_data=pd.read_csv('Labels.csv')
  label_data["text"] = label_data["text"].fillna('')
  label_data = label_data["text"]
  labels_data = []
  label_corpus = []
  for idx in range(len(label_data)):
    label = BeautifulSoup(label_data.iloc[idx])
    label = clean_str(label.get_text())
    label_corpus.append(label)
    labels_data.append(label)

  # Create Tokenizer
  tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(labels_data)

  # Zero Padding
  label_data = np.zeros((len(labels_data), MAX_SENT_LENGTH), dtype='int16')

  for j, sent in enumerate(labels_data):
      if j < MAX_SENTS:
          wordTokens = text_to_word_sequence(sent)
          k = 0
          for _, word in enumerate(wordTokens):
              if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                  label_data[j, k] = tokenizer.word_index[word]
                  k = k + 1

  label_word_index = tokenizer.word_index
###################### Preprocessing for Label #################################

###################### Preprocessing for Conent #################################

if PLUS_FLAG == True:
  data_train = pd.read_csv('Title+Body.csv')
elif TITLE_FLAG == True:
  data_train = pd.read_csv('Title.csv')
elif DESC_FLAG == True:
  data_train = pd.read_csv('Body.csv')

original_data_train = data_train
data_train["text"] = data_train["text"].fillna('')
data_train["text"] = data_train["text"].apply(lambda x: remove_url(x))
data_train["text"] = data_train["text"].apply(lambda x: remove_html(x))
data_train["text"] = data_train["text"].apply(lambda x: remove_emoji(x))
data_train["text"] = data_train["text"].apply(lambda text: remove_stopwords(text))

corpus_desc = []
labels = []
texts = []

for idx in range(data_train.text.shape[0]):
  text = BeautifulSoup(data_train.text[idx])
  text = clean_str(text.get_text())
  corpus_desc.append(text)
  sentences = tokenize.sent_tokenize(text)
  texts.append(sentences)
  labels.append(int(data_train.sentiment[idx]))

# Create Tokenizer
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(corpus_desc)

# Zero Padding
body_data = np.zeros((len(corpus_desc), MAX_SENTS, MAX_SENT_LENGTH), dtype='int16')

for i, sentences in enumerate(texts):
  for j, sent in enumerate(sentences):
    if j < MAX_SENTS:
      wordTokens = text_to_word_sequence(sent)
      k = 0
      for _, word in enumerate(wordTokens):
        if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
          body_data[i, j, k] = tokenizer.word_index[word]
          k = k + 1

word_index = tokenizer.word_index
labels = to_categorical(np.asarray(labels))
labels = to_categorical(np.asarray(labels[:,1]))


###################### Preprocessing for Content ################################

embedding_matrix_body = create_emb_mat(embedding_path, word_index, embedding_dim)

# Model Loading and Training

def training():
  
    ######################## layer for comment ##############################
    # Embedding layer
    if COMMENT_FLAG:
      embedding_layer_comment = Embedding(len(comment_word_index) + 1,
                                  embedding_dim,
                                  weights=[embedding_matrix_comment],
                                  mask_zero=False,
                                  input_length=MAX_SENT_LENGTH,
                                  trainable=True)

      comment_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
      embedded_sequences_comment = embedding_layer_comment(comment_input)
      l_lstm_comment = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences_comment)
      l_att_comment = Attention(embedding_dim)(l_lstm_comment)
      sent_encoder_comment = Model(comment_input, l_att_comment)

      comments_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
      comments_encoder = TimeDistributed(sent_encoder_comment)(comments_input)
      l_lstm_comments = Bidirectional(GRU(100, return_sequences=True))(comments_encoder)
      l_att_comments = Attention(embedding_dim)(l_lstm_comments)
    
    ######################## layer for command##############################
    # Embedding layer
    if COMMAND_FLAG:
      embedding_layer_command = Embedding(len(command_word_index) + 1,
                                  embedding_dim,
                                  weights=[embedding_matrix_command],
                                  mask_zero=False,
                                  input_length=MAX_SENT_LENGTH,
                                  trainable=True)

      command_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
      embedded_sequences_command = embedding_layer_command(command_input)
      l_lstm_command = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences_command)
      l_att_command = Attention(embedding_dim)(l_lstm_command)
      sent_encoder_command = Model(command_input, l_att_command)

      commands_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
      commands_encoder = TimeDistributed(sent_encoder_command)(commands_input)
      l_lstm_commands = Bidirectional(GRU(100, return_sequences=True))(commands_encoder)
      l_att_commands = Attention(embedding_dim)(l_lstm_commands)

    ######################## layer for code##############################
    # Embedding layer
    if CODE_FLAG:
      embedding_layer_code = Embedding(len(code_word_index) + 1,
                                  embedding_dim,
                                  weights=[embedding_matrix_code],
                                  mask_zero=False,
                                  input_length=MAX_SENT_LENGTH,
                                  trainable=True)

      code_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
      embedded_sequences_code = embedding_layer_code(code_input)
      l_lstm_code = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences_code)
      l_att_code = Attention(embedding_dim)(l_lstm_code)
      sent_encoder_code = Model(code_input, l_att_code)

      codes_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
      codes_encoder = TimeDistributed(sent_encoder_code)(codes_input)
      l_lstm_codes = Bidirectional(GRU(100, return_sequences=True))(codes_encoder)
      l_att_codes = Attention(embedding_dim)(l_lstm_codes)

    ######################## layer for label#############################
    if LABEL_FLAG:
      embedding_layer_label = Embedding(len(label_word_index) + 1,
                              embedding_dim,
                              weights=[embedding_matrix_label],
                              mask_zero=False,
                              input_length=MAX_SENT_LENGTH,
                              trainable=True)

      label_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
      embedded_sequences_label = embedding_layer_label(label_input)
      l_lstm_label = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences_label)
      l_att_label = Attention(embedding_dim)(l_lstm_label)

    ######################## layer for content##############################
    embedding_layer = Embedding(len(word_index) + 1,
                                embedding_dim,
                                weights=[embedding_matrix_body],
                                mask_zero=False,
                                input_length=MAX_SENT_LENGTH,
                                trainable=True)

    sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sentence_input)
    l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
    l_att = Attention(embedding_dim)(l_lstm)
    sent_encoder = Model(sentence_input, l_att)

    text_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
    text_encoder = TimeDistributed(sent_encoder)(text_input)
    l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(text_encoder)
    l_att_sent = Attention(embedding_dim)(l_lstm_sent)

    ################## Concatenate #######################
    concat_input_list = []
    model_input_list = []
    train_input_list = []
    test_input_list = []
    if COMMENT_FLAG:
      concat_input_list.append(l_att_comments)
      model_input_list.append(comments_input)
      train_input_list.append(x_comment_train)
      test_input_list.append(x_comment_test)
    if COMMAND_FLAG:
      concat_input_list.append(l_att_commands)
      model_input_list.append(commands_input)
      train_input_list.append(x_command_train)
      test_input_list.append(x_command_test)
    if CODE_FLAG:
      concat_input_list.append(l_att_codes)
      model_input_list.append(codes_input)
      train_input_list.append(x_code_train)
      test_input_list.append(x_code_test)
    if LABEL_FLAG:
      concat_input_list.append(l_att_label)
      model_input_list.append(label_input)
      train_input_list.append(x_label_train)
      test_input_list.append(x_label_test)
    concat_input_list.append(l_att_sent)
    model_input_list.append(text_input)
    train_input_list.append(x_body_train)
    test_input_list.append(x_body_test)
    
    if len(concat_input_list) > 1:
      concatenated = concatenate(concat_input_list)
    else:
      concatenated = concat_input_list[0]
    preds = Dense(2, activation='softmax')(concatenated)
    model = Model(inputs=model_input_list,outputs=[preds])
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', # or rmsprop
                  metrics=[tf_auc])
    # Stop when all epoches are run and return the result that has the best monitor metrics 
    early_stopping = EarlyStopping(monitor='val_loss',patience=EPOCH,restore_best_weights=True,verbose=0)
    callbacks=[early_stopping]
    model.fit(train_input_list, y_train, validation_split = 0.2,
              epochs=EPOCH, batch_size=batch_size,
              callbacks=[early_stopping])
    y_pred = model.predict(test_input_list).round().astype(int)
    y_true = y_test

    current_macro = f1_score(y_true, y_pred, average='macro')
    macro.append(current_macro)
    current_micro = f1_score(y_true, y_pred, average='micro')
    micro.append(current_micro)
    fpr, tpr, thresholds = roc_curve(y_true[:,1], y_pred[:,1], pos_label=1)
    current_auc = auc(fpr, tpr)
    auc_value.append(current_auc)
    cm = confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    using = original_data_train[['Number','sentiment']].iloc[test_index]
    idx = 0

    # initialize list of lists
    numbers_diff = []
    ytures_diff = []
    ypreds_diff = []
    numbers_same = []
    ytures_same = []
    ypreds_same = []

    for yture,ypred in zip([int(i) for i in y_true[:,1]],[int(i) for i in y_pred[:,1]]):
      if yture != ypred:
        numbers_diff.append(using['Number'].iloc[idx])
        ytures_diff.append(yture)
        ypreds_diff.append(ypred)
      if yture == ypred:
        numbers_same.append(using['Number'].iloc[idx])
        ytures_same.append(yture)
        ypreds_same.append(ypred)
      idx += 1

    data_diff = {'numbers_diff':numbers_diff,'ytrues_diff':ytures_diff,'ypreds_diff':ypreds_diff}
    data_same = {'numbers_same':numbers_same,'ytrues_same':ytures_same,'ypreds_same':ypreds_same}
    df_diff = pd.DataFrame(data_diff)
    df_same = pd.DataFrame(data_same)
    results = pd.Series([sum(macro)/len(macro),sum(micro)/len(micro),sum(auc_value)/len(auc_value)], name='results')
    df_diff = pd.concat([df_diff,results], axis=1)
    df_same = pd.concat([df_same,results], axis=1)
    # This will generate the result of false_neg&false_pos or true_neg&true_pos for one run
    df_diff.to_csv(f'../df_diff_{project}.csv')
    df_same.to_csv(f'../df_same_{project}.csv')
    return model

if __name__ == '__main__':
  macro = []
  micro = []
  auc_value = []
  df_log = pd.DataFrame(columns=['repeated_time','auc_list','Avg Macro F1', 'Avg Micro F1', 'Avg AUC'])
  for repeated_time in repeated_range:
    indices = np.arange(body_data.shape[0])
    train_index,test_index = train_test_split(indices, test_size=0.2, random_state=repeated_time)
    x_body_train, x_body_test = body_data[train_index], body_data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    if IMBLEARN == 'RandomOverSampler':
      smo = RandomOverSampler(random_state=666)
      nx1, ny1, nz1 = x_body_train.shape
      x_body_train = x_body_train.reshape((nx1,ny1*nz1))
      x_body_train, y_train = smo.fit_resample(x_body_train, y_train)
      temp_n, _ = x_body_train.shape
      x_body_train = x_body_train.reshape((temp_n,ny1,nz1))
      y_train = to_categorical(y_train.astype(int))
    
    if IMBLEARN == 'SMOTE':
      smo = SMOTE(random_state=666)
      nx1, ny1, nz1 = x_body_train.shape
      x_body_train = x_body_train.reshape((nx1,ny1*nz1))
      x_body_train, y_train = smo.fit_resample(x_body_train, y_train)
      temp_n, _ = x_body_train.shape
      x_body_train = x_body_train.reshape((temp_n,ny1,nz1))
      y_train = to_categorical(y_train.astype(int))

    if LABEL_FLAG:
      x_label_train, x_label_test = label_data[train_index], label_data[test_index]
      embedding_matrix_label = create_emb_mat(embedding_path, label_word_index, embedding_dim)
    if CODE_FLAG:
      x_code_train, x_code_test = code_data[train_index], code_data[test_index]
      embedding_matrix_code = create_emb_mat(embedding_path, code_word_index, embedding_dim)
    if COMMAND_FLAG:
      x_command_train, x_command_test = command_data[train_index], command_data[test_index]
      embedding_matrix_command = create_emb_mat(embedding_path, command_word_index, embedding_dim)
    if COMMENT_FLAG:
      x_comment_train, x_comment_test = comment_data[train_index], comment_data[test_index]
      embedding_matrix_comment = create_emb_mat(embedding_path, comment_word_index, embedding_dim)

    model = training()

    # this will generate the result
    new_row = {'repeated_time':repeated_time,'auc_list':str(auc_value),'Avg Macro F1':sum(macro) / len(macro), 'Avg Micro F1':sum(micro) / len(micro), 'Avg AUC':sum(auc_value) / len(auc_value)}
    df_log = df_log.append(new_row, ignore_index=True)
  df_log.to_csv(out_csv_name, mode='a', header=False)
                    